# 코드 고치기

┌─┬─┬─┬─┐

│0 │1 │2 │3 │

├─┼─┼─┼─┤

│4 │X │5 │6 │

├─┼─┼─┼─┤

│7 │8 │9 │10│

└─┴─┴─┴─┘

3 : 1점, 6 : -1점

0 왼 1 오 2 위 3 아래 

In [1]:
import numpy as np
import tensorflow as tf

# set parameters ###############################################################
initial_burning_epoch = 100
epoch = 5000
lr_rate = 1e-2
copy_period = 1
print_period = [10, 250]
epsilon_0 = 1e-4
# set parameters ###############################################################

# state
states = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
N_STATES = len(states)

# action
actions = [0, 1, 2, 3] 
N_ACTIONS = len(actions)

# transition probabilities
P = np.empty((N_STATES, N_ACTIONS, N_STATES))

#                0   1   2   3   4   5   6   7   8   9  10
P[ 0, 0, :] = [ .9,  0,  0,  0, .1,  0,  0,  0,  0,  0,  0]
P[ 0, 1, :] = [ .1, .8,  0,  0, .1,  0,  0,  0,  0,  0,  0]
P[ 0, 2, :] = [ .9, .1,  0,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 0, 3, :] = [ .1, .1,  0,  0, .8,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 1, 0, :] = [ .8, .2,  0,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 1, :] = [  0, .2, .8,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 2, :] = [ .1, .8, .1,  0,  0,  0,  0,  0,  0,  0,  0]
P[ 1, 3, :] = [ .1, .8, .1,  0,  0,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 2, 0, :] = [  0, .8, .1,  0,  0, .1,  0,  0,  0,  0,  0]
P[ 2, 1, :] = [  0,  0, .1, .8,  0, .1,  0,  0,  0,  0,  0]
P[ 2, 2, :] = [  0, .1, .8, .1,  0,  0,  0,  0,  0,  0,  0]
P[ 2, 3, :] = [  0, .1,  0, .1,  0, .8,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 3, 0, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 1, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 2, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]
P[ 3, 3, :] = [  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 4, 0, :] = [ .1,  0,  0,  0, .8,  0,  0, .1,  0,  0,  0]
P[ 4, 1, :] = [ .1,  0,  0,  0, .8,  0,  0, .1,  0,  0,  0]
P[ 4, 2, :] = [ .8,  0,  0,  0, .2,  0,  0,  0,  0,  0,  0]
P[ 4, 3, :] = [  0,  0,  0,  0, .2,  0,  0, .8,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 5, 0, :] = [  0,  0, .1,  0,  0, .8,  0,  0,  0, .1,  0]
P[ 5, 1, :] = [  0,  0, .1,  0,  0,  0, .8,  0,  0, .1,  0]
P[ 5, 2, :] = [  0,  0, .8,  0,  0, .1, .1,  0,  0,  0,  0]
P[ 5, 3, :] = [  0,  0,  0,  0,  0, .1, .1,  0,  0, .8,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 6, 0, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 1, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 2, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
P[ 6, 3, :] = [  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 7, 0, :] = [  0,  0,  0,  0, .1,  0,  0, .9,  0,  0,  0]
P[ 7, 1, :] = [  0,  0,  0,  0, .1,  0,  0, .1, .8,  0,  0]
P[ 7, 2, :] = [  0,  0,  0,  0, .8,  0,  0, .1, .1,  0,  0]
P[ 7, 3, :] = [  0,  0,  0,  0,  0,  0,  0, .9, .1,  0,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 8, 0, :] = [  0,  0,  0,  0,  0,  0,  0, .8, .2,  0,  0]
P[ 8, 1, :] = [  0,  0,  0,  0,  0,  0,  0,  0, .2, .8,  0]
P[ 8, 2, :] = [  0,  0,  0,  0,  0,  0,  0, .1, .8, .1,  0]
P[ 8, 3, :] = [  0,  0,  0,  0,  0,  0,  0, .1, .8, .1,  0]

#                0   1   2   3   4   5   6   7   8   9  10
P[ 9, 0, :] = [  0,  0,  0,  0,  0, .1,  0,  0, .8, .1,  0]
P[ 9, 1, :] = [  0,  0,  0,  0,  0, .1,  0,  0,  0, .1, .8]
P[ 9, 2, :] = [  0,  0,  0,  0,  0, .8,  0,  0, .1,  0, .1]
P[ 9, 3, :] = [  0,  0,  0,  0,  0,  0,  0,  0, .1, .8, .1]

#                0   1   2   3   4   5   6   7   8   9  10
P[10, 0, :] = [  0,  0,  0,  0,  0,  0, .1,  0,  0, .8, .1]
P[10, 1, :] = [  0,  0,  0,  0,  0,  0, .1,  0,  0,  0, .9]
P[10, 2, :] = [  0,  0,  0,  0,  0,  0, .8,  0,  0, .1, .1]
P[10, 3, :] = [  0,  0,  0,  0,  0,  0,  0,  0,  0, .1, .9]

# rewards
R = -0.02 * np.ones((N_STATES, N_ACTIONS)) 
R[3,:] = 1.
R[6,:] = -1.
    
# discount factor
gamma = 0.99

" TD "

state = tf.placeholder(tf.int32, shape=[None], name='state')
action = tf.placeholder(tf.int32, shape=[None], name='action')
reward = tf.placeholder(tf.float32, shape=[None], name='reward')
next_state = tf.placeholder(tf.int32, shape=[None], name='next_state')
next_action = tf.placeholder(tf.int32, shape=[None], name='next_action')

# https://www.tensorflow.org/api_docs/python/tf/one_hot
state_one_hot = tf.one_hot(state, N_STATES)
action_one_hot = tf.one_hot(action, N_ACTIONS)
next_state_one_hot = tf.one_hot(next_state, N_STATES)
next_action_one_hot = tf.one_hot(next_action, N_ACTIONS)

# https://www.tensorflow.org/api_docs/python/tf/concat
state_action_one_hot = tf.concat([state_one_hot, action_one_hot], 1)
next_state_action_one_hot = tf.concat([next_state_one_hot, next_action_one_hot], 1)

# state s: one hot encode
# action a: one hot encode
# Q[s,a]: [s as one hot encode, a as one hot encode] @ W 

with tf.variable_scope('main_net') as scope:
    W = tf.get_variable(name='W', \
                        shape=[N_STATES + N_ACTIONS, 1], \
                        dtype=tf.float32, \
                        initializer=tf.random_uniform_initializer(-1.0, 1.0))

with tf.variable_scope('target_net') as scope:
    W_target = tf.get_variable(name='W_target', \
                        shape=[N_STATES + N_ACTIONS, 1], \
                        dtype=tf.float32, \
                        initializer=tf.random_uniform_initializer(-1.0, 1.0))
    
# variable copy
from_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'main_net')
# [<tf.Variable 'main_net/W:0' shape=(15, 1) dtype=float32_ref>]
to_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'target_net')
# [<tf.Variable 'target_net/W_target:0' shape=(15, 1) dtype=float32_ref>]

copy_ops = [to_vars[i].assign(from_vars[i]) for i in range(len(from_vars))] #W_target에 W의 값을 대입한다는 것 같음.

# Previously for V
# V(S_t)\leftarrow V(S_t)+\alpha(R_{t+1}+\gamma V(S_{t+1})-V(S_t))
# V[s] += alpha * (R[s,a] + gamma * V[s1] - V[s])
# Q[s,a] += alpha * (R[s,a] + gamma * Q[s1,a1] - Q[s,a])

# Q target and error

AA = tf.placeholder(tf.float32, shape=[None], name='AA')

index = tf.placeholder(tf.int32, shape=[None], name='index')
each_action_one_hot = tf.one_hot(index, N_ACTIONS)
each_state_action_one_hot = tf.concat([next_state_one_hot, each_action_one_hot], 1)
KK=tf.matmul(each_state_action_one_hot, W)


Q_target = reward + gamma * AA

# Q_target = reward # this reward is actually reward + gamma * final_reward

Q = tf.matmul(state_action_one_hot, W)

error = Q_target - Q

loss = tf.reduce_mean(tf.square(error))
opt = tf.train.GradientDescentOptimizer(learning_rate=lr_rate)
train_ops = opt.minimize(loss, \
                         var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, \
                                                    'main_net'))

" Session "
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    print (" Initial Burning ")
    print("================================================") 
    mode = 0
    epsilon = 1.1            
    gradient_update_number = 0

    for this_ in range(epoch+initial_burning_epoch):
        
        if this_== initial_burning_epoch :
            mode = 1
            epsilon = epsilon_0
            print("================================================") 
            print (Q_out)
            print("================================================") 
            print (" Train ")
        
        this_epoch = this_ - mode * initial_burning_epoch
        
        if this_epoch % print_period[mode] == 0 : 
            print("================================================", this_epoch ) 
        
        # indicate game is not over yet
        done = False

        # choose initial state randomly, not from 3 or 6
        s = np.random.choice([0, 1, 2, 4, 5, 7, 8, 9, 10]) 
        
        if this_epoch % print_period[mode] == 0 :
                print('initial state :', s) 

        # this is policy update part of algorithm 
        # epsilon-greedy policy update
        # choose action using udated epsilon-greedy policy
        # if this part is replaced and if we choose action using current policy
        # this whole algorithm becomes TD, not SARSA, SARSA(lambda) nor Q-Learning

        while not done:
            
            Q_out = np.empty((N_STATES,N_ACTIONS))
            for i in range(N_STATES):
                for j in range(N_ACTIONS):
                    feed_dict = {state: [i], action: [j]}
                    Q_now = sess.run(Q, feed_dict=feed_dict)
                    Q_out[i,j] = Q_now[0][0]

                    
            # epsilon-greedy policy update
            if np.random.uniform(0.,1.) < epsilon:
                # random action selection
                a = np.random.choice(actions)
            else:
                # greedy action selection
                a = np.argmax(Q_out[s, :])
            
            
#             if gradient_update_number % copy_period == 0:
#                 sess.run(copy_ops)
                
            # choose next state using transition probabilities
            s1 = np.random.choice(states, p=P[s, a, :])
            if this_epoch % print_period[mode] == 0 :
                print("action : ", a, 'next state :', s1) 

            if s1 == 3:
                # if game is over, 
                # ready to break while loop by letting done = True
                done = True
                # train
                # reward =  R[s,a] + gamma*1
                aa = 1
                gradient_update_number += 1
                
            elif s1 == 6:
                # if game is over, 
                # ready to break while loop by letting done = True
                done = True
                # train
                # reward =  R[s,a] - gamma*1
                aa = -1
                gradient_update_number += 1
                               
            else:
                # if game is not over, continue playing game
                # train
                # reward =  R[s,a]
                
                AAA = np.zeros(N_ACTIONS)
                for i in range(N_ACTIONS) :
                    AAA[i] = Q_out [s1,i]
                aa = np.max(AAA)
                
                if this_epoch % print_period[mode] == 0 : 
                    print (AAA)

                gradient_update_number += 1
                
                              
            feed_dict = {state: [s], action: [a], reward: [R[s,a]], next_state: [s1], AA : [aa]}
            sess.run(train_ops, feed_dict=feed_dict)
            s = s1

                    
    print ("The End")
    print("================================================") 

    Q_final = np.empty((N_STATES,N_ACTIONS))
    for s in range(N_STATES):
        for a in range(N_ACTIONS):
            feed_dict = {state: [s], action: [a]}
            Q_now = sess.run(Q, feed_dict=feed_dict)
            Q_final[s,a] = Q_now[0][0]
    print(Q_final)

 Initial Burning 
================================================ 0
initial state : 0
action :  3 next state : 4
[ 1.11543369  0.69471526 -0.33979607  0.65440273]
action :  2 next state : 4
[ 1.11543369  0.69471526 -0.33979607  0.69166255]
action :  2 next state : 0
[-0.28041959 -0.70113802 -1.70716786 -0.70419073]
action :  0 next state : 0
[-0.28041959 -0.70113802 -1.7074635  -0.70419073]
action :  3 next state : 4
[ 1.14327562  0.72290111 -0.28342438  0.71984839]
action :  2 next state : 0
[-0.24477988 -0.6651544  -1.67147994 -0.63187957]
action :  2 next state : 1
[1.48981416 1.06943965 0.06353599 1.10271442]
action :  2 next state : 1
[1.48981416 1.06943965 0.12605548 1.10271442]
action :  3 next state : 0
[-0.18226039 -0.60263491 -1.51944184 -0.56936014]
action :  2 next state : 0
[-0.18226039 -0.60263491 -1.51944184 -0.59595472]
action :  3 next state : 4
[ 1.1436975   0.72332293 -0.16710395  0.73000312]
action :  1 next state : 0
[-0.12224364 -0.54261816 -1.43304503 -0.5023012

학습을 할 수록 한 방햐응로 고정되는 것을 볼 수 있음. 